Цель: В этом домашнем задании вам предстоит обойти все ловушки серверов, пробраться сквозь страницы html-кода и собрать себе свой собственный датасет.

По аналогии с занятием, возьмите интересующий вас сайт, на котором можно пособирать какие-то данные (и при этом API не предоставляется).
Напишите свой парсер, который будет бегать по страничкам и автоматически что-то собирать.

В качестве потенциальных целей для парсинга можно взять какие-нибудь блоги, выкачать оттуда публикации, авторов, число просмотром и комментариев. Можно посмотреть новостные ресурсы и скачать новостные статьи с их мета-информацией. Или любой другой интересный вам сайт.

Не забывайте, что парсинг - это ответственное мероприятие, поэтому не бомбардируйте несчастные сайты слишком частыми запросами (можно ограничить число запросов в секунду при помощи time.sleep(0.3), вставленного в теле цикла)

In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
import sys

from tqdm import tqdm_notebook

In [2]:
news_url = 'http://ratel.kz/raw/?page='

In [3]:
home_url = 'http://ratel.kz'

In [4]:
page = 1

In [5]:
requests.get(news_url+str(page))

<Response [200]>

In [6]:
news_page = requests.get(news_url+str(page))
news_page = news_page.content

In [7]:
news_page = BeautifulSoup(news_page, 'lxml')

In [8]:
news_page = news_page.find('div', attrs={'class':'col-sm-8 col-sm-push-1'})

In [9]:
def get_articles(news_page):
    articles = news_page.findAll('div', attrs={'class':'article article_list'})
    return articles

In [10]:
articles = get_articles(news_page)

In [11]:
def get_article(news_page):
    article = news_page.find('div', attrs={'class':'article article_list'})
    return article

#### Заголовки новостей

In [12]:
def get_header(article):
    header = article.find('div', attrs = {'class':'article__title'})
    header = header.text.strip()
    return header

In [13]:
get_header(news_page)

'Владельцам кафе и ресторанов Талдыкоргана начали выписывать штрафы'

In [14]:
for article in articles:
    print(get_header(article))

Владельцам кафе и ресторанов Талдыкоргана начали выписывать штрафы
Очередные слухи о путях  распространения коронавируса обсуждают в Талдыкоргане
Производители антисептика связаны по рукам
О карантине, медицинских масках, продуктах и лекарствах рассказал на брифинге вице-премьер РК
О возрастном цензе заболевших COVID-19  в Нур-Султане сообщили в горздраве столицы
В Караганде у двух пациентов обнаружили коронавирус
Денсаулық министрлігі Қарағандыда кімдер коронавирус жұқтырғанын айтып берді
Елбасы Нурсултан Назарбаев обратился к казахстанцам
Субъекты малого и среднего бизнеса смогут получить льготный кредит сроком на год
Уже одиннадцать очагов заражения коронавирусом установили в различных частях Алматы
Танки в городе
Information on passenger flights to/from Nur-Sultan and Almaty
Актюбинского адвоката заподозрили в мошенничестве
Акимат Усть-Каменогорска судится с подрядчиком из-за долгостроя
Инфекция ошақтары туралы Алматыда тараған ақпарат жалған – әкімдік
Задержание председателя суда 

#### Короткий текст статьи

In [15]:
def get_short_text(article):
    short_text = article.find('div', attrs = {'class':'article__text'}).text
    short_text = short_text.strip()
    return short_text

In [16]:
for article in articles:
    print(get_short_text(article))

На самых злостных нарушителей будут заводить уголовные дела
На этот раз в поле зрения горожан попал  местный аэропорт
Из-за бюрократии в СКО не могут начать продавать востребованный в условиях ЧП отечественный антисептик
Ералы Тугжанов отметил, что 70% чиновников уже перешли на дистанционную работу
Также стали известны сроки внедрения экспресс-тестов на коронавирус
Оба заражённых – пассажиры рейса "Минск-Нур-Султан". Карантин вводить не будут
Алматы қаласында жиырма екі адам, Қарағандыда екі, Нұр-Сұлтанда жиырма бес адамда коронавирус анықталды
Он призвал поддержать действия президента страны Касым-Жомарта Токаева и правительства РК
Глава государства  объявил о новых мерах поддержки малого и среднего бизнеса в условиях ЧП
Все очаги в настоящее время берут под контроль
Скопление бронетехники около Аягоза напугало восточно-казахстанцев
The Civil Aviation Committee recommends to passengers do not travel to the airports of Nur-Sultan and Almaty starting from 00:00 on March 22
Он пообещал з

#### Страничка одной статьи

In [17]:
def get_link(article):
    link = article.find('a', attrs= {'class':'article__img-link'}).get('href')
    return link

In [18]:
for article in articles:
    print(get_link(article))

/raw/vladeltsam_kafe_i_restoranov_taldykorgana_nachali_vypisyvat_shtrafy
/raw/ocherednye_sluhi_o_putjah_rasprostranenija_koronavirusa_obsuzhdajut_v_taldykorgane
/raw/proizvoditeli_antiseptika_svjazany_po_rukam
/raw/o_karantine_meditsinskih_maskah_produktah_i_lekarstvah_rasskazal_na_brifinge_vitse_premer_rk_
/raw/o_vozrastnom_tsenze_zabolevshih_covid_19_v_nur_sultane_soobschili_v_gorzdrave_stolitsy
/raw/v_karagande_u_dvuh_patsientov_obnaruzhili_koronavirus
/raw/densauly_ministrligi_araandyda_kimder_koronavirus_zhtyranyn_ajtyp_berdi
/raw/elbasy_nursultan_nazarbaev_obratilsja_k_kazahstantsam
/raw/subekty_malogo_i_srednego_biznesa_smogut_poluchit_lgotnyj_kredit_srokom_na_god
/raw/uzhe_odinnadtsat_ochagov_zarazhenija_koronavirusom_ustanovili_v_razlichnyh_chastjah_almaty
/raw/tanki_v_gorode
/raw/information_on_passenger_flights_tofrom_nur_sultan_and_almaty
/raw/aktjubinskogo_advokata_zapodozrili_v_moshennichestve
/raw/akimat_ust_kamenogorska_suditsja_s_podrjadchikom_iz_za_dolgostroja
/raw/in

In [22]:
article = get_article(news_page)

In [19]:
def get_subpage(link):
    sub_page = requests.get(home_url+str(link)).content
    sub_page = BeautifulSoup(sub_page, 'lxml')
    return sub_page

In [23]:
news_link = get_link(article)

In [25]:
sub_page = get_subpage(news_link)

#### Просмотры статьи

In [26]:
def get_views(sub_page):
    post_views = sub_page.find('div', attrs={'class':'post__views'})
    text = post_views.text
    views = text.strip().split()[0]
    return int(views)

In [27]:
get_views(sub_page)

102

#### Дата публикации

In [28]:
import datetime
import locale
locale.setlocale(locale.LC_TIME, 'ru_RU.UTF-8')

'ru_RU.UTF-8'

In [29]:
# def get_date(news_page):
#     pattern = re.compile(r'[\d]{1,2}[\s]{1}[А-я]{3}[\s]{1}[\d]{4}')
#     text = news_page.find('div', attrs = {'class':'article__date'}).text
#     date = re.match(pattern, text).group(0)
#     return date if date else ""

# get_date(news_page)

In [30]:
def get_datetime(article):
    article_info = article.find('div', attrs = {'class':'article__date'}).text
    time_data = article_info.split(" — ")[0]
    date = datetime.datetime.strptime(time_data, '%d %b %Y, %H:%M')
    return date

In [31]:
for article in articles:
    print(get_datetime(article))

2020-03-20 16:20:00
2020-03-20 15:20:00
2020-03-20 14:35:00
2020-03-20 14:30:00
2020-03-20 13:50:00
2020-03-20 12:35:00
2020-03-20 11:00:00
2020-03-20 09:05:00
2020-03-20 08:35:00
2020-03-20 08:05:00
2020-03-19 18:40:00
2020-03-19 18:30:00
2020-03-19 18:00:00
2020-03-19 17:30:00
2020-03-19 17:00:00
2020-03-19 16:20:00


https://stackoverflow.com/questions/41839678/how-to-use-russian-date-string-with-strptime


#### Автор статьм

In [32]:
article = news_page.find('div', attrs= {'class':'article__date'})

In [33]:
def get_author(article):
    article_info = article.find('div', attrs = {'class':'article__date'}).text
    article_info = article_info.split(" — ")
    author = article_info[1].strip()
    return author

In [34]:
for article in articles:
    print(get_author(article))

Алия АХМЕДИЕВА
Алия АХМЕДИЕВА
Ольга ВАЙТОВИЧ
Ратель
Асем  МИРЖЕКЕЕВА
Анастасия БАГРОВА
Ратель
Ратель
Ратель
Ратель
Антон СЕРГИЕНКО
Ратель
Дмитрий МАТВЕЕВ
Антон СЕРГИЕНКО
Ратель
Ольга ВОРОНЬКО


###  Получим все статьи с одной новостной странички

In [39]:
def get_article_data(article):
    news_link = get_link(article)
    sub_page = get_subpage(news_link)
    article_data = {
        'header': get_header(article),
        'author': get_author(article),
        'date': get_datetime(article),
        'views': get_views(sub_page),
        'short_text': get_short_text(article)
    }
    return article_data

In [40]:
def get_news(page):
    news_page = requests.get(news_url+str(page))
    news_page = news_page.content
    news_page = BeautifulSoup(news_page, 'lxml')  
    news_page = news_page.find('div', attrs={'class':'col-sm-8 col-sm-push-1'})
    return news_page

In [41]:
news_page = get_news(1)
articles = get_articles(news_page)

for article in articles:
    article_data = get_article_data(article)
    print(article_data['header'])
    print(f"Написано: {article_data['author']}, Опубликовано: {article_data['date']}")
    print(article_data['views'])
    print('====================')
    time.sleep(0.3)

1 миллион тенге выплатит чиновник, взявший 1 тысячу за бесплатную госуслугу
Написано: Антон СЕРГИЕНКО, Опубликовано: 2020-03-20 16:50:00
23
Владельцам кафе и ресторанов Талдыкоргана начали выписывать штрафы
Написано: Алия АХМЕДИЕВА, Опубликовано: 2020-03-20 16:20:00
104
Очередные слухи о путях  распространения коронавируса обсуждают в Талдыкоргане
Написано: Алия АХМЕДИЕВА, Опубликовано: 2020-03-20 15:20:00
161
Производители антисептика связаны по рукам
Написано: Ольга ВАЙТОВИЧ, Опубликовано: 2020-03-20 14:35:00
331
О карантине, медицинских масках, продуктах и лекарствах рассказал на брифинге вице-премьер РК
Написано: Ратель, Опубликовано: 2020-03-20 14:30:00
141
О возрастном цензе заболевших COVID-19  в Нур-Султане сообщили в горздраве столицы
Написано: Асем  МИРЖЕКЕЕВА, Опубликовано: 2020-03-20 13:50:00
620
В Караганде у двух пациентов обнаружили коронавирус
Написано: Анастасия БАГРОВА, Опубликовано: 2020-03-20 12:35:00
583
Денсаулық министрлігі Қарағандыда кімдер коронавирус жұқтырға

## Parsing start

In [42]:
news = pd.DataFrame(columns=['header', 'author', 'date',  'views', 'short_text'])

In [43]:
total_pages = 3

In [44]:
# for page_number in tqdm_notebook(range(total_pages)):

#     news_page = get_news(page_number)
#     articles = get_articles(news_page)
    
#     for article in tqdm_notebook(articles, leave=False):
#         try:
#             article_data = get_article_data(article)
#             news = news.append(article_data, ignore_index=True)
#             time.sleep(0.3)
#         except:
#             pass

HBox(children=(IntProgress(value=0, max=3), HTML(value='')))

HBox(children=(IntProgress(value=0, max=16), HTML(value='')))

HBox(children=(IntProgress(value=0, max=16), HTML(value='')))

HBox(children=(IntProgress(value=0, max=16), HTML(value='')))

In [ ]:
for page_number in range(total_pages):

    news_page = get_news(page_number)
    articles = get_articles(news_page)
    
    for article in articles:
        try:
            article_data = get_article_data(article)
            news = news.append(article_data, ignore_index=True)
            time.sleep(0.3)
        except:
            pass

In [45]:
news.head()

,header,author,date,views,short_text
0,"1 миллион тенге выплатит чиновник, взявший 1 т...",Антон СЕРГИЕНКО,2020-03-20 16:50:00,27,"Житель Семея, решивший отблагодарить государст..."
1,Владельцам кафе и ресторанов Талдыкоргана нача...,Алия АХМЕДИЕВА,2020-03-20 16:20:00,117,На самых злостных нарушителей будут заводить у...
2,Очередные слухи о путях распространения корон...,Алия АХМЕДИЕВА,2020-03-20 15:20:00,163,На этот раз в поле зрения горожан попал местн...
3,Производители антисептика связаны по рукам,Ольга ВАЙТОВИЧ,2020-03-20 14:35:00,334,Из-за бюрократии в СКО не могут начать продава...
4,"О карантине, медицинских масках, продуктах и л...",Ратель,2020-03-20 14:30:00,142,"Ералы Тугжанов отметил, что 70% чиновников уже..."


### Analytics

In [46]:
news = news.drop_duplicates().reset_index(drop=True)

In [47]:
news.views.astype(int).hist();

In [49]:
news[news.views == max(news.views)]

,header,author,date,views,short_text
34,"27-летний актюбинец ""заработал"" на ажиотаже 1 ...",Дмитрий МАТВЕЕВ,2020-03-19 15:15:00,51602,В областном центре задержали спекулянта медици...


In [50]:
news.loc[34].header

'27-летний актюбинец "заработал" на ажиотаже 1 млн 800 тыс. тенге'

In [51]:
news.loc[34].short_text

'В областном центре задержали спекулянта медицинскими масками'